In [140]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [141]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

# Pull Gas Prices from AAA.com

In [142]:
# Visit the AAA site
url = 'https://gasprices.aaa.com/'
browser.visit(url)

# Delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)

False

In [143]:
prices_data = []
fuel_types = []

# Convert the base url visited to html
prices_soup = soup(browser.html, 'html.parser')

#Search for the objects that contain the fuel types and the fuel prices.
table_elem = prices_soup.find('table', class_='table-mob')
title_elem = table_elem.find_all('th')
prices_elem = table_elem.find('tbody').find_all('tr')[0].find_all('td')

# Extract the text for fuel types
for item in title_elem:
    if item.get_text() != '':
        fuel_types.append(item.get_text())
        
# Extract the text for fuel prices
for item in prices_elem:
    if item.get_text()[0] == '$':
        prices_data.append(item.get_text())

# Combine the two lists into a dictionary.
prices_dict = dict(zip(fuel_types, prices_data))
print(prices_dict)
browser.quit()

{'Regular': '$3.355', 'Mid-Grade': '$3.809', 'Premium': '$4.125', 'Diesel': '$5.031', 'E85': '$2.751'}


# Download & Transform Historical Prices

In [202]:
crude_hist_df = pd.read_csv('./Resources/crude_price_history.csv')
gas_hist_df = pd.read_csv('./Resources/gas_price_history.csv', header=2)
gas_hist_df.head()

,Date,Weekly U.S. All Grades All Formulations Retail Gasoline Prices (Dollars per Gallon),Weekly U.S. All Grades Conventional Retail Gasoline Prices (Dollars per Gallon),Weekly U.S. All Grades Reformulated Retail Gasoline Prices (Dollars per Gallon),Weekly U.S. Regular All Formulations Retail Gasoline Prices (Dollars per Gallon),Weekly U.S. Regular Conventional Retail Gasoline Prices (Dollars per Gallon),Weekly U.S. Regular Reformulated Retail Gasoline Prices (Dollars per Gallon),Weekly U.S. Midgrade All Formulations Retail Gasoline Prices (Dollars per Gallon),Weekly U.S. Midgrade Conventional Retail Gasoline Prices (Dollars per Gallon),Weekly U.S. Midgrade Reformulated Retail Gasoline Prices (Dollars per Gallon),Weekly U.S. Premium All Formulations Retail Gasoline Prices (Dollars per Gallon),Weekly U.S. Premium Conventional Retail Gasoline Prices (Dollars per Gallon),Weekly U.S. Premium Reformulated Retail Gasoline Prices (Dollars per Gallon),Weekly U.S. No 2 Diesel Retail Prices (Dollars per Gallon),Weekly U.S. No 2 Diesel Ultra Low Sulfur (0-15 ppm) Retail Prices (Dollars per Gallon),Weekly U.S. No 2 Diesel Low Sulfur (15-500 ppm) Retail Prices (Dollars per Gallon),Unnamed: 16
0,"Aug 20, 1990",NaN,NaN,NaN,1.191,1.191,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Aug 27, 1990",NaN,NaN,NaN,1.245,1.245,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Sep 03, 1990",NaN,NaN,NaN,1.242,1.242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Sep 10, 1990",NaN,NaN,NaN,1.252,1.252,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Sep 17, 1990",NaN,NaN,NaN,1.266,1.266,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [203]:
gas_hist_df.dtypes

Date                                                                                        object
Weekly U.S. All Grades All Formulations Retail Gasoline Prices  (Dollars per Gallon)       float64
Weekly U.S. All Grades Conventional Retail Gasoline Prices  (Dollars per Gallon)           float64
Weekly U.S. All Grades Reformulated Retail Gasoline Prices  (Dollars per Gallon)           float64
Weekly U.S. Regular All Formulations Retail Gasoline Prices  (Dollars per Gallon)          float64
Weekly U.S. Regular Conventional Retail Gasoline Prices  (Dollars per Gallon)              float64
Weekly U.S. Regular Reformulated Retail Gasoline Prices  (Dollars per Gallon)              float64
Weekly U.S. Midgrade All Formulations Retail Gasoline Prices  (Dollars per Gallon)         float64
Weekly U.S. Midgrade Conventional Retail Gasoline Prices  (Dollars per Gallon)             float64
Weekly U.S. Midgrade Reformulated Retail Gasoline Prices  (Dollars per Gallon)             float64
Weekly U.S

In [204]:
crude_hist_df.dtypes

Date      object
Open     float64
High     float64
Low      float64
Close    float64
dtype: object

In [205]:
gas_hist_df['Date'] = pd.to_datetime(gas_hist_df['Date'])
gas_hist_df['Date'].dropna()

0      1990-08-20
1      1990-08-27
2      1990-09-03
3      1990-09-10
4      1990-09-17
          ...    
1681   2022-11-07
1682   2022-11-14
1683   2022-11-21
1684   2022-11-28
1685   2022-12-05
Name: Date, Length: 1686, dtype: datetime64[ns]

In [206]:
gas_hist_df.drop(['Unnamed: 16', 'Weekly U.S. No 2 Diesel Low Sulfur (15-500 ppm) Retail Prices  (Dollars per Gallon)'], axis=1, inplace=True)

In [207]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
now = datetime.now()
monday = now - timedelta(days = now.weekday()) - relativedelta(years=1)
print(monday)

2021-12-05 14:09:16.516947


In [208]:
gas_hist_filtered = gas_hist_df[gas_hist_df['Date'] >= monday]
gas_hist_filtered.head()

,Date,Weekly U.S. All Grades All Formulations Retail Gasoline Prices (Dollars per Gallon),Weekly U.S. All Grades Conventional Retail Gasoline Prices (Dollars per Gallon),Weekly U.S. All Grades Reformulated Retail Gasoline Prices (Dollars per Gallon),Weekly U.S. Regular All Formulations Retail Gasoline Prices (Dollars per Gallon),Weekly U.S. Regular Conventional Retail Gasoline Prices (Dollars per Gallon),Weekly U.S. Regular Reformulated Retail Gasoline Prices (Dollars per Gallon),Weekly U.S. Midgrade All Formulations Retail Gasoline Prices (Dollars per Gallon),Weekly U.S. Midgrade Conventional Retail Gasoline Prices (Dollars per Gallon),Weekly U.S. Midgrade Reformulated Retail Gasoline Prices (Dollars per Gallon),Weekly U.S. Premium All Formulations Retail Gasoline Prices (Dollars per Gallon),Weekly U.S. Premium Conventional Retail Gasoline Prices (Dollars per Gallon),Weekly U.S. Premium Reformulated Retail Gasoline Prices (Dollars per Gallon),Weekly U.S. No 2 Diesel Retail Prices (Dollars per Gallon),Weekly U.S. No 2 Diesel Ultra Low Sulfur (0-15 ppm) Retail Prices (Dollars per Gallon)
1633,2021-12-06,3.440,3.279,3.761,3.341,3.204,3.639,3.800,3.587,4.120,4.063,3.875,4.288,3.674,3.674
1634,2021-12-13,3.414,3.252,3.738,3.315,3.178,3.614,3.776,3.558,4.106,4.039,3.846,4.270,3.649,3.649
1635,2021-12-20,3.395,3.229,3.727,3.295,3.154,3.602,3.763,3.539,4.099,4.024,3.826,4.261,3.626,3.626
1636,2021-12-27,3.375,3.211,3.704,3.275,3.136,3.577,3.743,3.521,4.078,4.008,3.810,4.244,3.615,3.615
1637,2022-01-03,3.381,3.216,3.710,3.281,3.141,3.585,3.746,3.524,4.079,4.012,3.813,4.249,3.613,3.613


In [209]:
gas_hist_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53 entries, 1633 to 1685
Data columns (total 15 columns):
 #   Column                                                                                   Non-Null Count  Dtype         
---  ------                                                                                   --------------  -----         
 0   Date                                                                                     53 non-null     datetime64[ns]
 1   Weekly U.S. All Grades All Formulations Retail Gasoline Prices  (Dollars per Gallon)     53 non-null     float64       
 2   Weekly U.S. All Grades Conventional Retail Gasoline Prices  (Dollars per Gallon)         53 non-null     float64       
 3   Weekly U.S. All Grades Reformulated Retail Gasoline Prices  (Dollars per Gallon)         53 non-null     float64       
 4   Weekly U.S. Regular All Formulations Retail Gasoline Prices  (Dollars per Gallon)        53 non-null     float64       
 5   Weekly U.S. 

In [210]:
from pymongo import MongoClient
client = MongoClient("mongodb://localhost:27017/") 
db = client['gas_app']
col = db.current_pricing

col.update_one({}, {"$set": prices_dict}, upsert=True)

In [211]:
gas_hist_filtered.reset_index(inplace=True)
gas_hist_filtered.drop(['index'], axis=1, inplace=True)
gas_hist_filtered.index = gas_hist_filtered.index.map(str)

C:\Users\Baldo\anaconda3\envs\PyData\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [212]:
col = db.gas_history
gas_dict = gas_hist_filtered.to_dict()
col.update_one({}, {"$set": gas_dict}, upsert=True)

In [216]:
crude_hist_df.index = crude_hist_df.index.map(str)
crude_dict = crude_hist_df.to_dict()
crude_dict

{'Date': {'0': '12/05/2022',
  '1': '12/02/2022',
  '2': '12/01/2022',
  '3': '11/30/2022',
  '4': '11/29/2022',
  '5': '11/28/2022',
  '6': '11/25/2022',
  '7': '11/23/2022',
  '8': '11/22/2022',
  '9': '11/21/2022',
  '10': '11/18/2022',
  '11': '11/17/2022',
  '12': '11/16/2022',
  '13': '11/15/2022',
  '14': '11/14/2022',
  '15': '11/11/2022',
  '16': '11/10/2022',
  '17': '11/09/2022',
  '18': '11/08/2022',
  '19': '11/07/2022',
  '20': '11/04/2022',
  '21': '11/03/2022',
  '22': '11/02/2022',
  '23': '11/01/2022',
  '24': '10/31/2022',
  '25': '10/28/2022',
  '26': '10/27/2022',
  '27': '10/26/2022',
  '28': '10/25/2022',
  '29': '10/24/2022',
  '30': '10/21/2022',
  '31': '10/20/2022',
  '32': '10/19/2022',
  '33': '10/18/2022',
  '34': '10/17/2022',
  '35': '10/14/2022',
  '36': '10/13/2022',
  '37': '10/12/2022',
  '38': '10/11/2022',
  '39': '10/10/2022',
  '40': '10/07/2022',
  '41': '10/06/2022',
  '42': '10/05/2022',
  '43': '10/04/2022',
  '44': '10/03/2022',
  '45': '09/

In [217]:
col = db.crude_history
col.update_one({}, {"$set": crude_dict}, upsert=True)